# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize, download
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm.auto import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import re
import time
import json

In [115]:
stemmer = SnowballStemmer("russian")
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(
    ['это', 'как', 'так', 'и', 'в', 'над', 'к', 'до', 'не', 'на', 'но', 'за', 'то', 'с', 'ли', 'а', 'во', 'от', 'со',
     'для', 'о', 'же', 'ну', 'вы', 'бы', 'что', 'кто', 'он', 'она', 'оно', 'из-за', 'также'])

In [117]:
top_tokens_to_remove = [
    "год",
    "года",
    "align",
    "дата",
    "small",
    "году",
    "br",
    "флаг",
    "access",
    "group",
    "го",
    "место",
    "archive",
    "гс",
    "style",
    "род",
    "время",
    "website",
    "of",
    "ru",
    "страницы",
    "автор",
    "the",
    "заглавие",
    "медаль",
    "составляет",
    "sfn",
    "nbsp",
    "ibge",
    "чел",
    "лет",
    "ширина",
    "который",
    "сайт",
    "вид",
    "площадь",
    "её",
    "no",
    "png",
    "тип",
    "флагификация",
    "состав",
    "de",
    "января",
    "ссылка",
    "нп",
    "географии",
    "июля",
    "декабря",
    "список",
    "background",
    "марта",
    "isbn",
    "left",
    "мая",
    "октября",
    "апреля",
    "сентября",
    "utc",
    "бс",
    "июня",
    "годы",
    "text",
    "do",
    "стал",
    "века",
    "августа",
    "svg",
    "герой",
    "издание",
    "нл",
    "клуб",
    "deadlink",
    "лист",
    "ноября",
    "февраля",
    "ii",
    "archiveurl",
    "per",
    "capita",
    "которые",
    "страниц",
    "id",
    "штат",
    "день",
    "ссылки",
    "см",
    "согласно"
]

In [118]:
specific_stopwords = [
    'категория', 'категории',
    'примечания', 'примечание',
    'изображение', 'изображение',
    'файл', 'файлы',
    'автонумерация',
    'столбцов', 'столбец',
    'название',
    'оформление',
    'заголовок',
    'автоссылка',
    'выравнивание',
    'сортировка', 'сортировки',
    'текст',
    'число',
    'цвет',
    'км²',
    'lang-pt',
    'px',
    'lat',
    'deg',
    'min',
    'sec',
    'lon',
    'dir',
    'ref',
    'name',
    'cite',
    'web',
    'url',
    'title',
    'publisher',
    'date',
    'accessdate',
    'lang',
    'pt',
    'ref',
    'author',
    'publisher',
    'en',
    'lang-en',
    'jpg',
    'noinclude',
    'xa0',
    'center',
    'right',
    'gallery'
    
]

# Предобработка данных

In [119]:
def text_cleaning(text):
  # Удаляем вики-разметку с учетом шаблонов и параметров
    text = re.sub(r'{{.*?}}', '', text, flags=re.DOTALL)
  # Удаляем HTML-теги
    text = re.sub(r'<.*?>', '', text)
  # Удаляем ссылки с учетом различных форматов
    text = re.sub(r'\[\[([^|\]]*\|)?([^\]]*)\]\]', r'\2', text)
    text = re.sub(r'(?<=[^\w\d])-|-(?=[^\w\d])|[^\w\d\s-]', '', text) #Удаляем все символы, кроме букв, цифр, пробелов и дефисов
    text = re.sub(r"\d+px", "", text)
    text = re.sub(r"\b[a-z]{3}_[a-z]+( \d+)?\b", "", text)

  # Удаляем лишние пробелы и переводы строк
    text = re.sub(r'\s+', ' ', text).strip()
  # Приводим к нижнему регистру
    text = text.lower()
    
    return text

In [120]:
def remove_stopwords(text): # удаление стоп-слов
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in specific_stopwords and token not in top_tokens_to_remove]
    return " ".join(filtered_tokens)

In [121]:
def stemming(texts):
    stemmed_texts_list = []
    for text in tqdm(texts.split()):
        try:
            tokens = word_tokenize(text)
            stemmed_tokens = [stemmer.stem(token) for token in tokens]
            text = " ".join(stemmed_tokens)
            stemmed_texts_list.append(text)

        except Exception as e:
            print(e)

    new_text = " ".join(stemmed_texts_list)
    return new_text

In [160]:
def remove_non_russian_words(text): # удаление слов на латинице
    russian_word_pattern = re.compile(r'^[а-яА-Я]+$')
    words = text.split()
    russian_words = [word for word in words if russian_word_pattern.match(word)]
    return ' '.join(russian_words)

In [161]:
def preprocessing_data(text):
    text = text_cleaning(text)
    text = remove_stopwords(text)
    text = remove_non_russian_words(text)
    #text = stemming(text)
    
    return text
    
    

In [131]:
data = pd.read_csv('texts_csv.csv')

In [228]:
with open('emotional_data_list.pickle', 'wb') as file:
    pickle.dump(TOXIC_TOKENS, file)

In [230]:
data.info() # 5 записей без текста

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29254 entries, 0 to 29258
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29254 non-null  object
 1   type    29254 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 571.4+ KB


In [231]:
data = data[data.text.isna() == False]

In [133]:
data = data[['text', 'type']]

In [110]:
data.type.unique() # 22 вида статей

array(["['Статья не проходила экспертную проверку', 'Обычная статья']",
       "['Готовая статья']",
       "['Готовая статья', 'Статья не проходила экспертную проверку']",
       "['Готовая статья', 'Статья успешно прошла экспертную проверку']",
       "['Хорошая статья', 'Статья успешно прошла экспертную проверку']",
       "['Обычная статья']",
       "['Избранная статья', 'Статья успешно прошла экспертную проверку']",
       "['Готовая статья', 'Кандидат в избранные статьи']",
       "['Избранная статья']",
       "['Готовая статья', 'Кандидат в хорошие статьи']",
       "['Хорошая статья']",
       "['Статья под удаление', 'Статья не проходила экспертную проверку']",
       "['Статья успешно прошла экспертную проверку']",
       "['Готовая статья', 'Статья не успешно прошла экспертную проверку']",
       "{'0': 'Готовая статья', '1': 'Кандидат в хорошие статьи', '3': 'Статья успешно прошла экспертную проверку'}",
       "['Хорошая статья', 'Статья не проходила экспертную проверку'

In [111]:
GOOD_TYPES = [
       "['Готовая статья']",
       "['Готовая статья', 'Статья успешно прошла экспертную проверку']",
       "['Хорошая статья', 'Статья успешно прошла экспертную проверку']",
       "['Избранная статья', 'Статья успешно прошла экспертную проверку']",
       "['Готовая статья', 'Кандидат в избранные статьи']",
       "['Избранная статья']",
       "['Готовая статья', 'Кандидат в хорошие статьи']",
       "['Хорошая статья']",
       "['Статья успешно прошла экспертную проверку']",
       "{'0': 'Готовая статья', '1': 'Кандидат в хорошие статьи', '3': 'Статья успешно прошла экспертную проверку'}",
       "['Кандидат в хорошие статьи', 'Готовая статья', 'Статья успешно прошла экспертную проверку']",
       "{'0': 'Готовая статья', '1': 'Кандидат в избранные статьи', '3': 'Статья успешно прошла экспертную проверку'}",
       "['Кандидат в хорошие статьи', 'Готовая статья']",
       "['Готовая статья', 'Кандидат в хорошие статьи', 'Статья успешно прошла экспертную проверку']",
       "['Кандидат в избранные статьи', 'Готовая статья', 'Хорошая статья']"
]

BAD_TYPES = [
       "['Статья не проходила экспертную проверку', 'Обычная статья']",
       "['Готовая статья', 'Статья не проходила экспертную проверку']",
       "['Обычная статья']",
       "['Статья под удаление', 'Статья не проходила экспертную проверку']",
       "['Готовая статья', 'Статья не успешно прошла экспертную проверку']",
       "['Хорошая статья', 'Статья не проходила экспертную проверку']",
       "['Статья не успешно прошла экспертную проверку']",
]

In [84]:
data['target'] = data['type'].apply(lambda x: 1 if x in GOOD_TYPES else 0) #1 если статья нейтральна и 0 в ином случае

In [85]:
(data[data.target == 1].count()) / (data[data.target == 0].count()) # bad types намного больше

text      0.142913
type      0.142913
target    0.142913
dtype: float64

In [134]:
%%time
data['text'] = data['text'].apply(preprocessing_data)

CPU times: total: 2min 14s
Wall time: 2min 16s


Найдем самые часто встречающиеся слова, которые также следует удалить: стоп-слова, слова-паразиты, слова на латинице, служебные токены

In [81]:
mes_text_corpus = []
for value_list in data.text:
    tokens = word_tokenize(value_list)
    mes_text_corpus.extend(tokens)

In [83]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(mes_text_corpus)
vocabulary = vectorizer.get_feature_names_out()

In [84]:
token_counts = X.sum(axis=0)
token_freq = {token: count for token, count in zip(vocabulary, token_counts.tolist()[0])}

In [98]:
sorted_tokens = sorted(token_freq.items(), key=lambda x: x[1], reverse=True)
top_tokens = sorted_tokens[:200]  # Первые 10 самых частых токенов

# ---------------------------------------------------------------------------------------------

In [ ]:
data.to_csv('latest_data.csv') #сохраним новый датасет, который отправим на разметку в gemini